<h1 align="center">Net Present Value Monte Carlo Simulation</h1>


# Import all Required Libraries.

In [43]:
# random provides shuffle() that shuffles the original list in place, and sample() that returns a new list that is randomly shuffled
import random

import csv

# Help Functions

In [2]:
def numberDiff(num1: float, num2: float) -> float:
    diff: float
    if num1 > num2:
        diff = num1 - num2
    else:
        diff = num2 - num1
    return diff

# Cash Flow Objects

## Cash Flow Object Definition

The cash flows that an investment will generate are estimated. One cash flow can range from the best to the normal to the worst case. The distribution can be defined for each cash flow.

In [3]:
class cashFlowObject:
    bestCase_CashFlow: float
    bestCase_CashFlow_Discounted: float
    bestCase_Probability: float
    averageCase_CashFlow: float
    averageCase_CashFlow_Discounted: float
    averageCase_Probability: float
    worstCase_CashFlow: float
    worstCase_CashFlow_Discounted: float
    worstCase_Probability: float
    cashFlow_Period: int
    name: str
        
    def __init__(self, bestCase_CashFlow: float, bestCase_Probability: float, averageCase_CashFlow: float, averageCase_Probability: float, worstCase_CashFlow: float, worstCase_Probability: float, cashFlow_Period: float, name: str):
        self.bestCase_CashFlow = bestCase_CashFlow
        self.bestCase_Probability = bestCase_Probability
        self.averageCase_CashFlow = averageCase_CashFlow
        self.averageCase_Probability = averageCase_Probability
        self.worstCase_CashFlow = worstCase_CashFlow
        self.worstCase_Probability = worstCase_Probability
        self.cashFlow_Period = cashFlow_Period
        self.name = name
    

## Define all Cash Flows

In [4]:
cashFlows: list[cashFlowObject]
cashFlows = []

In [5]:
cashFlows.append(cashFlowObject(-20.0, 0.1, -22.0, 0.7, -24.0, 0.2, 0, "Parameter 1"))
cashFlows.append(cashFlowObject(-12.0, 0.08, -14.0, 0.8, -16.0, 0.12, 0, "Parameter 2"))
cashFlows.append(cashFlowObject(-3,0.05,-2,0.83,-1,0.12,1,"Parameter 3"))
cashFlows.append(cashFlowObject(-2,0.08,-3,0.78,-4,0.14,1,"Parameter 4"))
cashFlows.append(cashFlowObject(-1,0.04,-2,0.86,-3,0.1,1,"Parameter 5"))
cashFlows.append(cashFlowObject(19,0.1,18,0.75,17,0.15,1,"Parameter 6"))
cashFlows.append(cashFlowObject(11,0.08,10,0.76,7,0.16,1,"Parameter 7"))
cashFlows.append(cashFlowObject(-4,0.06,-3,0.81,-2,0.13,2,"Parameter 3"))
cashFlows.append(cashFlowObject(-3,0.09,-4,0.76,-5,0.15,2,"Parameter 4"))
cashFlows.append(cashFlowObject(-2,0.05,-3,0.84,-4,0.11,2,"Parameter 5"))
cashFlows.append(cashFlowObject(22,0.11,21,0.73,20,0.16,2,"Parameter 6"))
cashFlows.append(cashFlowObject(12,0.09,11,0.74,9,0.17,2,"Parameter 7"))

## Discount all Cash Flows

In [6]:
discountFactror_WACC = 0.12

In [7]:
for cashFlowObject in cashFlows:
    cashFlowObject.bestCase_CashFlow_Discounted = round(cashFlowObject.bestCase_CashFlow * (1+discountFactror_WACC)**(-cashFlowObject.cashFlow_Period),2)
    cashFlowObject.averageCase_CashFlow_Discounted = round(cashFlowObject.averageCase_CashFlow * (1+discountFactror_WACC)**(-cashFlowObject.cashFlow_Period),2)
    cashFlowObject.worstCase_CashFlow_Discounted = round(cashFlowObject.worstCase_CashFlow * (1+discountFactror_WACC)**(-cashFlowObject.cashFlow_Period),2)

# Create Urns for every Cash Flow Object

The urns contain the possible values of a cash flow object. </br>
</br>
How to fill the urns?
1. The values and probabilities between the cases are evenly distributed. </br>
2. In order to get an representative distribution in every urn, the evenly distributed values are  multiplied with the representative and evenly distributed probabilities.

In [8]:
class cashFlowUrn:
    valueDistribution: list[float]
    cashFlow_Period: int
    name: str
    
    def __init__(self, cashFlow_Period: int, name: str):
        self.valueDistribution = []
        self.cashFlow_Period = cashFlow_Period
        self.name = name

In [9]:
evenlyDistributionStepsFromCaseToCase: int = 50

In [21]:
cashFlowUrns: list[cashFlowUrn] = []
amountOfCoveredCases: int = 1

for cfo in cashFlows:
    # create urn for every cash flow object
    curCashFlowUrn = cashFlowUrn(cfo.cashFlow_Period, cfo.name)
    
    # calc parameters for the distribution of the urn values
    valueStepsFromWCtoAV = (numberDiff(cfo.averageCase_CashFlow_Discounted, cfo.worstCase_CashFlow_Discounted) / evenlyDistributionStepsFromCaseToCase)
    valueStepsFromACtoBC = (numberDiff(cfo.bestCase_CashFlow_Discounted, cfo.averageCase_CashFlow_Discounted) / evenlyDistributionStepsFromCaseToCase)
    percentStepsFromWCtoAV = (numberDiff(cfo.averageCase_Probability, cfo.worstCase_Probability) / evenlyDistributionStepsFromCaseToCase)
    percentStepsFromACtoBC = (numberDiff(cfo.averageCase_Probability, cfo.bestCase_Probability) / evenlyDistributionStepsFromCaseToCase)
    
    # Init parameter for the loops to fill the urn
    curValue: float
    curDistribution: float
    
    # Add worst case (inclusive) to average case (inclusive) values into the urn
    for w2a in range(0, evenlyDistributionStepsFromCaseToCase + 1):
        curValue = None
        curDistribution = None
        if cfo.worstCase_CashFlow_Discounted <= cfo.averageCase_CashFlow_Discounted: 
            curValue = cfo.worstCase_CashFlow_Discounted + (w2a * valueStepsFromWCtoAV)
        else:
            curValue = cfo.worstCase_CashFlow_Discounted - (w2a * valueStepsFromWCtoAV)
        if cfo.worstCase_Probability <= cfo.averageCase_Probability:
            curDistribution = (cfo.worstCase_Probability + (w2a * percentStepsFromWCtoAV)) * 100
        else:
            curDistribution = (cfo.worstCase_Probability - (w2a * percentStepsFromWCtoAV)) * 100
        # fill the urn
        curValue = round(curValue,4)
        for w2aFill in range(0, int(round(curDistribution))):
            curCashFlowUrn.valueDistribution.append(curValue)
        
    # Add average case (exclusive) to beste case (inclusive) values into the urn
    for a2b in range(1, evenlyDistributionStepsFromCaseToCase + 1):
        curValue = None
        curDistribution = None
        if cfo.averageCase_CashFlow_Discounted <= cfo.bestCase_CashFlow_Discounted:
            curValue = cfo.averageCase_CashFlow_Discounted + (a2b * valueStepsFromACtoBC)
        else: 
            curValue = cfo.averageCase_CashFlow_Discounted - (a2b * valueStepsFromACtoBC)
        if cfo.averageCase_Probability <= cfo.bestCase_Probability:
             curDistribution = (cfo.averageCase_Probability + (a2b * percentStepsFromACtoBC)) * 100
        else:
            curDistribution = (cfo.averageCase_Probability - (a2b * percentStepsFromACtoBC)) * 100
        curValue = round(curValue,4)
        # fill the urn
        for a2bFill in range(0, int(round(curDistribution))):
            curCashFlowUrn.valueDistribution.append(curValue)
            
    cashFlowUrns.append(curCashFlowUrn)
    
    print(curCashFlowUrn.name, "& t=", curCashFlowUrn.cashFlow_Period, "; Possibilities (elements in urn):", f"{len(curCashFlowUrn.valueDistribution):,}")
    amountOfCoveredCases = amountOfCoveredCases * len(curCashFlowUrn.valueDistribution)


print("Number of urns:", f"{len(cashFlowUrns):,}")
print("Amount of possible cases due to configuration:", f"{amountOfCoveredCases:,}")

Parameter 1 & t= 0 ; Possibilities (elements in urn): 4,265
Parameter 2 & t= 0 ; Possibilities (elements in urn): 4,510
Parameter 3 & t= 1 ; Possibilities (elements in urn): 4,584
Parameter 4 & t= 1 ; Possibilities (elements in urn): 4,461
Parameter 5 & t= 1 ; Possibilities (elements in urn): 4,657
Parameter 6 & t= 1 ; Possibilities (elements in urn): 4,386
Parameter 7 & t= 1 ; Possibilities (elements in urn): 4,412
Parameter 3 & t= 2 ; Possibilities (elements in urn): 4,544
Parameter 4 & t= 2 ; Possibilities (elements in urn): 4,411
Parameter 5 & t= 2 ; Possibilities (elements in urn): 4,608
Parameter 6 & t= 2 ; Possibilities (elements in urn): 4,338
Parameter 7 & t= 2 ; Possibilities (elements in urn): 4,362
Number of urns: 12
Amount of possible cases due to configuration: 61,950,645,862,095,899,584,537,363,635,489,320,258,764,800


Shuffle all urns to improve the random draw.

In [22]:
shuffleFrequency: int = 100

In [23]:
for urn in cashFlowUrns:
    for shuffleCounter in range(0, shuffleFrequency):
        random.shuffle(urn.valueDistribution) 

# Simulate Possible Net Present Values

In [24]:
possibleNPVs: list[float]

Help function

In [25]:
def getRandomNPV() -> float:
    randomNPV: float = 0
    for curUrn in cashFlowUrns:
        randomNPV += random.choice(curUrn.valueDistribution)
    return randomNPV

Parameter

Amount of simulations (cases) - must be smaller than the amount of possible cases due to configuration.

In [26]:
print("Amount of possible cases due to configuration:", f"{amountOfCoveredCases:,}")

Amount of possible cases due to configuration: 61,950,645,862,095,899,584,537,363,635,489,320,258,764,800


In [47]:
simulationCases: int = 1000000

In [48]:
possibleNPVs = []
for simulation in range(0, simulationCases):
    possibleNPVs.append(getRandomNPV())

In [49]:
print(len(possibleNPVs))

1000000


In [52]:
import csv
with open('simulationValues.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for el in possibleNPVs:
        writer.writerow([round(el,2)])